### РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ

#### Факультет физико-математических и естественных наук  
#### Кафедра математического моделирования и искусственного интеллекта

## ОТЧЕТ ПО ЛАБОРАТОРНОЙ РАБОТЕ № 2

#### Дисциплина: Интеллектуальный анализ данных

##### Студент:  Матюшкин Денис Владимирович
##### Группа:   НПИбд-02-21

## Москва 2024
***

### Вариант № 4

Алгоритм: Apriori

День недели (поле order_dow таблицы orders): “3”

Код департамента (поле department_id таблицы products): “18”

Запрос: определить час дня, в который было совершено более всего покупок (заказов)

Показатель оценки ассоциативных правил: рычаг (leverage)

1. При помощи модуля sqlite3 откройте базу данных Instacart в файле instacart.db.

In [80]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [81]:
import sqlite3
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [5]:
conn = sqlite3.connect('instacart.db')
cursor = conn.cursor()

type(cursor)

sqlite3.Cursor

2. Загрузите таблицы departments и products в датафреймы Pandas. При помощи запроса SELECT извлеките из таблицы order_products__train записи, соответствующие указанным в индивидуальном задании дню недели (поле order_dow таблицы orders) и коду департамента (поле department_id таблицы products) и загрузите в датафрейм Pandas. Определите количество строк в полученном датафрейме, количество транзакций (покупок) и определите количество товаров (столбец product_id) в транзакциях датафрейма.

In [6]:
departments = pd.read_sql_query("SELECT * FROM departments", conn)
products = pd.read_sql_query("SELECT * FROM products", conn)

In [7]:
departments

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [43]:
products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [33]:
query = """
SELECT opt.*
FROM order_products__train opt
JOIN orders o ON opt.order_id = o.order_id
JOIN products p ON opt.product_id = p.product_id
WHERE o.order_dow = 3 AND p.department_id = 18
"""

In [34]:
df = pd.read_sql_query(query, conn)
df

,order_id,product_id,add_to_cart_order,reordered
0,208213,14491,10,0
1,208213,16384,11,0
2,208213,17622,2,0
3,208213,34061,8,0
4,208213,46807,3,0
...,...,...,...,...
1769,769520,8912,10,1
1770,197679,30192,9,0
1771,197679,43989,1,0
1772,197679,44497,2,0


In [35]:
print(f"Количество строк: {len(df)}")
print(f"Количество транзакций: {df['order_id'].nunique()}")
print(f"Количество товаров: {df['product_id'].nunique()}")

Количество строк: 1774
Количество транзакций: 772
Количество товаров: 474


Берем уникальные данные при выводе:

3. Выполните к датафрейму запрос, указанный в индивидуальном задании.

In [38]:
query_hour = """
SELECT opt.order_id, o.order_hour_of_day
FROM order_products__train opt
JOIN orders o ON opt.order_id = o.order_id
JOIN products p ON opt.product_id = p.product_id
WHERE o.order_dow = 3 AND p.department_id = 18;
"""

In [39]:
df_hour = pd.read_sql_query(query_hour, conn)
df_hour

,order_id,order_hour_of_day
0,208213,12
1,208213,12
2,208213,12
3,208213,12
4,208213,12
...,...,...
1769,769520,10
1770,197679,11
1771,197679,11
1772,197679,11


In [41]:
purchases = df_hour.groupby('order_hour_of_day').size()

max_hour = purchases.idxmax()
max_purchases = purchases.max()

print(f"Час дня с наибольшим количеством покупок: {max_hour}")
print(f"Количество покупок в этот час: {max_purchases}")

Час дня с наибольшим количеством покупок: 10
Количество покупок в этот час: 164


4. Постройте транзакционную базу данных из полученного датафрейма, используя в качестве идентификатора транзакции столбец order_id, а в качестве названий товаров - поле product_name из датафрейма для таблицы products, соответствующее столбцу product_id. Найдите в транзакционной базе данных транзакцию с наибольшим количеством товаров и выведите ее на экран.

In [49]:
df_merged = pd.merge(df, products[['product_id', 'product_name']], on='product_id', how='left')
transaction_db = df_merged.groupby('order_id')['product_name'].apply(list).reset_index()
transaction_db

,order_id,product_name
0,1003391,[Sunny Days Strawberry Snack Bars]
1,1009266,"[Organic Pears, Apples, Peaches, Pumpkin + Cin..."
2,10208,[Kids Sensible Foods Broccoli Littles]
3,1021481,"[Organic Whole Grain Oatmeal Cereal Baby Food,..."
4,1022467,[Organic Fiber & Protein Pear Blueberry & Spin...
...,...,...
767,991937,"[Apples, Kale & Avocados Organic Baby Food, Pe..."
768,992347,[Simply Clean Baby Wipes]
769,99567,"[Training Pants Learning Designs 4T-5T, Wipes]"
770,996153,"[Organic Peas & Brown Rice Stage 2 Baby Food, ..."


In [54]:
transaction_db['num_products'] = transaction_db['product_name'].apply(len)
max_tran = transaction_db.loc[transaction_db['num_products'].idxmax()]

print(f"ID с наибольшим количеством товаров: {max_tran['order_id']}")
print(f"Кол-во товаров: {max_tran['num_products']}")
print(f"Список товаров: {max_tran['product_name']}")

ID с наибольшим количеством товаров: 784984
Кол-во товаров: 18
Список товаров: ['Apple Blueberry Fruit Yogurt Smoothie', 'Organic Whole Grain Oatmeal Cereal Baby Food', 'Strawberry and Banana Fruit Puree', 'Pasta Pick-ups - Cheese Ravioli', 'Organic Toddler Puree Blueberry Superfruit', 'Graduates Grabbers Fruit & Yogurt Strawberry Banana', 'Organic Jr Apple Cinnamon Oatmeal', 'Organic Strawberry Banana Fruit Yogurt Smoothie', "3rd Foods Chicken in Broth & Noodle Dinner with Lil' Bits", 'Mighty 4 Essential Tots Spinach, Kiwi, Barley & Greek Yogurt Nutrition Blend', 'Organic Stage 3 Pear Carrot Apricot Baby Food', 'Carrot and Broccoli Baby Food Puree', "Sesame Street Organic Original Cruchin' Cracker", 'Purple Carrot & blueberry Puffs', 'Sunny Days Strawberry Snack Bars', 'Tender Chicken & Stars Stage 3', 'Simply Clean Fragrance Free Wipes', 'Graduates Lil Crunchies Veggie Dip']


5. Постройте по транзакционной базе данных бинарную базу данных в формате датафрейма пакета mlxtend. По бинарной базе данных определите и выведите на экран три наиболее популярных товара и определите количество покупок (транзакций) этих товаров.


In [58]:
te = TransactionEncoder()
transaction_bin = te.fit(transaction_db['product_name']).transform(transaction_db['product_name'])
binary_df = pd.DataFrame(transaction_bin, columns=te.columns_)
binary_df

,1F Organic Oatmeal Cereal Organic Cereal,1pk 270ct Refill,1st Foods Applesauce Baby Food,1st Foods Baby Food- Pears,1st Foods Bananas Baby Food,1st Foods Green Beans,1st Foods Peaches,1st Foods Prunes,1st Foods Squash,1st Foods Sweet Potatoes,...,"Veggie & Fruit Snacks, Carrot, Mango & Orange",Veggie Pick Ups Diced Carrots,White Buttermints,Whole Grain Rice Cereal,Wholesome Breakfast Apple Raisin Baby Food,Wholesome Breakfast Blueberry Banana,Wipes,Yellow Rice & Chicken with Vegetables in Sauce with Green Beans & Carrots,"Yum-A-Roo's Organic Toddler Snacks Pea, Banana, & Pineapple",Zucchini Banana & Amaranth Organic Baby Food
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
767,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
768,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
769,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
770,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Вывод трех наиболее популярных товара и количество их покупок:

In [65]:
counts = binary_df.sum().sort_values(ascending=False)
counts.head(3)

,0
Baby Food Stage 2 Blueberry Pear & Purple Carrot,39
Gluten Free SpongeBob Spinach Littles,38
Spinach Peas & Pear Stage 2 Baby Food,36


6. При помощи указанного в индивидуальном задании метода построения популярных наборов предметов постройте и выведите на экран популярный набор предметов с (абсолютной) минимальной поддержкой не менее 3, имеющий максимальную длину. При отсутствии такого набора уменьшите поддержку до 2. В случае нехватки вычислительных ресурсов (слишком долгой работы программы) при построении популярного набора предметов сокращайте число записей в наборе данных (например, делая выборку половины записей набора).

In [71]:
min_support = 3 / len(binary_df)
itemsets_apr = apriori(binary_df, min_support=min_support, use_colnames=True)
itemsets_apr

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.005181,(AdvanceCare Oral Electrolyte Solution - Cherr...
1,0.003886,(All-One Baby Unscented Pure-Castile Bar Soap)
2,0.003886,(Apple & Cherry Simple Combos Organic)
3,0.007772,(Apple Blueberry Fruit Yogurt Smoothie)
4,0.007772,(Apple Peach Oatmeal Stage 2)
...,...,...
272,0.003886,"(Organic Strawberry Yogurt & Fruit Snack, Bana..."
273,0.003886,"(Chunky Blend Vegetable Beef Pilaf Baby Food, ..."
274,0.003886,"(Chunky Blend Vegetable Beef Pilaf Baby Food, ..."
275,0.005181,"(Mighty 4 Sweet Potato, Blueberry, Millet & Gr..."


In [72]:
itemsets_apr['length'] = itemsets_apr['itemsets'].apply(lambda x: len(x))
itemsets_apr

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets,length
0,0.005181,(AdvanceCare Oral Electrolyte Solution - Cherr...,1
1,0.003886,(All-One Baby Unscented Pure-Castile Bar Soap),1
2,0.003886,(Apple & Cherry Simple Combos Organic),1
3,0.007772,(Apple Blueberry Fruit Yogurt Smoothie),1
4,0.007772,(Apple Peach Oatmeal Stage 2),1
...,...,...,...
272,0.003886,"(Organic Strawberry Yogurt & Fruit Snack, Bana...",3
273,0.003886,"(Chunky Blend Vegetable Beef Pilaf Baby Food, ...",3
274,0.003886,"(Chunky Blend Vegetable Beef Pilaf Baby Food, ...",3
275,0.005181,"(Mighty 4 Sweet Potato, Blueberry, Millet & Gr...",3


Популярные наборы товаров с максимальной длиной и поддержкой не менее 3:

In [74]:
max_length = itemsets_apr[itemsets_apr['length'] == itemsets_apr['length'].max()]
max_length

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets,length
260,0.003886,(Organic Strawberry Banana Fruit Yogurt Smooth...,3
261,0.003886,"(Apple and Carrot Stage 2 Baby Food, Baby Food...",3
262,0.007772,"(Apple and Carrot Stage 2 Baby Food, Spinach P...",3
263,0.003886,"(Apple and Carrot Stage 2 Baby Food, Baby Food...",3
264,0.003886,"(Apple and Carrot Stage 2 Baby Food, Peach, A...",3
265,0.003886,"(Pear & Mango Stage 2 Baby Food, Apple and Car...",3
266,0.003886,"(Baby Food Pears Squash, Apples, Kale & Avocad...",3
267,0.005181,"(Apples, Kale & Avocados Organic Baby Food, Sp...",3
268,0.005181,"(Baby Food Stage 2 Pumpkin Banana, Spinach Pea...",3
269,0.006477,"(Spinach Peas & Pear Stage 2 Baby Food, Brocco...",3


7. Используя пакет mlxtend или реализацию на Python, постройте набор ассоциативных правил для полученного популярного наборов предметов с уровнем достоверности (confidence) не ниже $0.6$.

In [85]:
rules = association_rules(itemsets_apr, metric="confidence", min_threshold=0.6)
rules

/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Apple Blueberry Fruit Yogurt Smoothie),(Organic Strawberry Banana Fruit Yogurt Smoothie),0.007772,0.016839,0.005181,0.666667,39.589744,0.005050,2.949482,0.982376
1,(Baby Food Pears Squash),"(Apples, Kale & Avocados Organic Baby Food)",0.006477,0.020725,0.003886,0.600000,28.950000,0.003752,2.448187,0.971751
2,(Baby Food Pears Squash),"(Apples, Pumpkin & Carrots Organic Baby Food)",0.006477,0.009067,0.005181,0.800000,88.228571,0.005123,4.954663,0.995111
3,(Baby Food Pears Squash),"(Bananas, Raspberries & Oats Organic Baby Food)",0.006477,0.011658,0.003886,0.600000,51.466667,0.003811,2.470855,0.986962
4,(Chunky Blend Vegetable Beef Pilaf Baby Food),(Organic Stage 3 Spaghetti with Cheese 6 Ounce...,0.005181,0.006477,0.003886,0.750000,115.800000,0.003852,3.974093,0.996528
5,(Organic Stage 3 Spaghetti with Cheese 6 Ounce...,(Chunky Blend Vegetable Beef Pilaf Baby Food),0.006477,0.005181,0.003886,0.600000,115.800000,0.003852,2.487047,0.997827
6,(Chunky Blend Vegetable Beef Pilaf Baby Food),(Tender Chicken & Stars Stage 3),0.005181,0.010363,0.005181,1.000000,96.500000,0.005128,inf,0.994792
7,(Chunky Blend Vegetable Beef Pilaf Baby Food),(Vegetable Chicken Soup Stage 3),0.005181,0.003886,0.003886,0.750000,193.000000,0.003866,3.984456,1.000000
8,(Vegetable Chicken Soup Stage 3),(Chunky Blend Vegetable Beef Pilaf Baby Food),0.003886,0.005181,0.003886,1.000000,193.000000,0.003866,inf,0.998700
9,"(Mighty 4 Essential Tots Spinach, Kiwi, Barley...","(Mighty 4 Sweet Potato, Blueberry, Millet & Gr...",0.010363,0.015544,0.006477,0.625000,40.208333,0.006316,2.625216,0.985340


8. Для построенного набора ассоциативных правил вычислите показатель (меру) оценки ассоциативных правил, указанную в индивидуальном задании, и выведите на экран ассоциативное правило, имеющее наилучшее значение этого показателя оценки.

Показатель оценки ассоциативных правил leverage уже вычислен и находится в rules. Он вычесляется по формуле:

$$ leverage(A→T) = P(A \vee T) - support(A) * support(T) $$

[Источник](https://ranalytics.github.io/data-mining/054-Association-Rules-Algos.html)

In [86]:
rules.loc[rules['leverage'].idxmax()]

,12
antecedents,(Organic Fruit Yogurt Smoothie Mixed Berry)
consequents,(Organic Strawberry Banana Fruit Yogurt Smoothie)
antecedent support,0.009067
consequent support,0.016839
support,0.007772
confidence,0.857143
lift,50.901099
leverage,0.007619
conviction,6.882124
zhangs_metric,0.989325


In [91]:
rules.sort_values(by='leverage', ascending=False).head(2)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
12,(Organic Fruit Yogurt Smoothie Mixed Berry),(Organic Strawberry Banana Fruit Yogurt Smoothie),0.009067,0.016839,0.007772,0.857143,50.901099,0.007619,6.882124,0.989325
25,"(Apple and Carrot Stage 2 Baby Food, Baby Food...",(Spinach Peas & Pear Stage 2 Baby Food),0.010363,0.046632,0.007772,0.750000,16.083333,0.007289,3.813472,0.947644
